In [44]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import sklearn as skl

In [45]:
#import data 
raw_df = pd.read_csv('nba_rankings_2014-2020.csv', index_col=False)
raw_df.head()

,Unnamed: 0,PLAYER,TEAM,AGE,SEASON,GP,W,L,MIN,PTS,...,AST,TOV,STL,BLK,PF,FP,DD2,TD3,+/-,rankings
0,0,Aaron Gordon,ORL,24,2019-20,62,30,32,32.5,14.4,...,3.7,1.6,0.8,0.6,2.0,31.9,20.0,1.0,-1.1,80.0
1,1,Aaron Holiday,IND,23,2019-20,66,42,24,24.5,9.5,...,3.4,1.3,0.8,0.2,1.8,19.3,3.0,0.0,1.7,76.0
2,2,Abdel Nader,OKC,26,2019-20,55,37,18,15.8,6.3,...,0.7,0.8,0.4,0.4,1.4,11.1,0.0,0.0,-1.5,71.0
3,3,Adam Mokoka,CHI,21,2019-20,11,3,8,10.2,2.9,...,0.4,0.2,0.4,0.0,1.5,5.5,0.0,0.0,4.5,68.0
4,4,Admiral Schofield,WAS,23,2019-20,33,9,24,11.2,3.0,...,0.5,0.2,0.2,0.1,1.5,6.3,0.0,0.0,-1.7,71.0


In [46]:
raw_df['SEASON'].value_counts()

2019-20    507
2017-18    418
2016-17    393
2014-15    366
2018-19    364
2015-16    364
Name: SEASON, dtype: int64

In [47]:
season_list = raw_df["SEASON"].nunique()
enc = OneHotEncoder(sparse=False)
encode_df = pd.DataFrame(enc.fit_transform(raw_df.SEASON.values.reshape(-1,1)))

In [48]:
encode_df.columns = enc.get_feature_names(['SEASON'])
encode_df.head()



/Users/mollysprecher/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,SEASON_2014-15,SEASON_2015-16,SEASON_2016-17,SEASON_2017-18,SEASON_2018-19,SEASON_2019-20
0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,1.0


In [49]:

new_df = raw_df.merge(encode_df, left_index=True, right_index=True).drop("SEASON",1)
new_df.head()

/Users/mollysprecher/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


,Unnamed: 0,PLAYER,TEAM,AGE,GP,W,L,MIN,PTS,FGM,...,DD2,TD3,+/-,rankings,SEASON_2014-15,SEASON_2015-16,SEASON_2016-17,SEASON_2017-18,SEASON_2018-19,SEASON_2019-20
0,0,Aaron Gordon,ORL,24,62,30,32,32.5,14.4,5.4,...,20.0,1.0,-1.1,80.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,Aaron Holiday,IND,23,66,42,24,24.5,9.5,3.5,...,3.0,0.0,1.7,76.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2,Abdel Nader,OKC,26,55,37,18,15.8,6.3,2.2,...,0.0,0.0,-1.5,71.0,0.0,0.0,0.0,0.0,0.0,1.0
3,3,Adam Mokoka,CHI,21,11,3,8,10.2,2.9,1.1,...,0.0,0.0,4.5,68.0,0.0,0.0,0.0,0.0,0.0,1.0
4,4,Admiral Schofield,WAS,23,33,9,24,11.2,3.0,1.1,...,0.0,0.0,-1.7,71.0,0.0,0.0,0.0,0.0,0.0,1.0


In [50]:
#define features 
X = new_df.copy()
X = X.drop("rankings", axis=1)
X = X.drop("PLAYER", axis=1)
X = X.drop("TEAM", axis=1)
X.head()

,Unnamed: 0,AGE,GP,W,L,MIN,PTS,FGM,FGA,FG%,...,FP,DD2,TD3,+/-,SEASON_2014-15,SEASON_2015-16,SEASON_2016-17,SEASON_2017-18,SEASON_2018-19,SEASON_2019-20
0,0,24,62,30,32,32.5,14.4,5.4,12.4,43.7,...,31.9,20.0,1.0,-1.1,0.0,0.0,0.0,0.0,0.0,1.0
1,1,23,66,42,24,24.5,9.5,3.5,8.5,41.4,...,19.3,3.0,0.0,1.7,0.0,0.0,0.0,0.0,0.0,1.0
2,2,26,55,37,18,15.8,6.3,2.2,4.8,46.8,...,11.1,0.0,0.0,-1.5,0.0,0.0,0.0,0.0,0.0,1.0
3,3,21,11,3,8,10.2,2.9,1.1,2.5,42.9,...,5.5,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,1.0
4,4,23,33,9,24,11.2,3.0,1.1,2.8,38.0,...,6.3,0.0,0.0,-1.7,0.0,0.0,0.0,0.0,0.0,1.0


In [11]:
raw_df.dtypes

Unnamed: 0      int64
PLAYER         object
TEAM           object
AGE             int64
SEASON         object
GP              int64
W               int64
L               int64
MIN           float64
PTS           float64
FGM           float64
FGA           float64
FG%           float64
3PM           float64
3PA           float64
3P%           float64
FTM           float64
FTA           float64
FT%           float64
OREB          float64
DREB          float64
REB           float64
AST           float64
TOV           float64
STL           float64
BLK           float64
PF            float64
FP            float64
DD2           float64
TD3           float64
+/-           float64
rankings      float64
dtype: object

In [51]:
#define target 
y = new_df["rankings"].ravel()
y[:5]

array([80., 76., 71., 68., 71.])

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=14)

In [53]:
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [54]:
rf_model = RandomForestClassifier(n_estimators=128, random_state=14)

In [55]:
rf_model = rf_model.fit(X_train_scaled, y_train)

In [57]:
predictions = rf_model.predict(X_test_scaled)
predictions

array([90., 73., 76., 76., 72., 87., 74., 81., 83., 76., 77., 77., 72.,
       72., 74., 76., 78., 78., 70., 70., 75., 75., 68., 72., 75., 73.,
       75., 76., 71., 72., 81., 72., 70., 74., 77., 81., 76., 75., 73.,
       80., 78., 68., 86., 70., 76., 77., 85., 76., 73., 79., 71., 73.,
       85., 81., 87., 74., 72., 75., 75., 90., 76., 70., 69., 72., 76.,
       89., 77., 74., 79., 70., 72., 87., 76., 72., 71., 87., 74., 72.,
       74., 76., 73., 76., 69., 89., 76., 71., 72., 68., 68., 76., 74.,
       74., 76., 74., 78., 72., 75., 70., 70., 73., 78., 90., 77., 69.,
       97., 76., 87., 77., 76., 86., 76., 77., 77., 77., 76., 74., 72.,
       76., 73., 73., 79., 78., 79., 68., 85., 72., 78., 87., 79., 86.,
       79., 79., 74., 74., 72., 75., 96., 75., 76., 73., 73., 79., 70.,
       71., 72., 75., 86., 94., 76., 78., 73., 75., 73., 76., 76., 78.,
       75., 77., 72., 74., 73., 77., 73., 86., 76., 77., 79., 85., 77.,
       74., 75., 71., 79., 74., 77., 72., 79., 75., 82., 72., 71

In [60]:
acc_score = accuracy_score(y_test, predictions)
acc_score

0.20563847429519072

In [68]:
comparison_df = pd.DataFrame(predictions, columns=["predictions"])
comparison_df["real"] = y_test
comparison_df.head(20)

,predictions,real
0,90.0,95.0
1,73.0,77.0
2,76.0,77.0
3,76.0,75.0
4,72.0,73.0
5,87.0,84.0
6,74.0,72.0
7,81.0,84.0
8,83.0,82.0
9,76.0,75.0


In [61]:
classification_report(y_test, predictions)

/Users/mollysprecher/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/mollysprecher/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/mollysprecher/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modi

'              precision    recall  f1-score   support\n\n        65.0       0.00      0.00      0.00         6\n        66.0       0.00      0.00      0.00         5\n        67.0       0.50      0.12      0.20         8\n        68.0       0.32      0.35      0.33        17\n        69.0       0.17      0.17      0.17        18\n        70.0       0.26      0.27      0.26        33\n        71.0       0.17      0.19      0.18        21\n        72.0       0.21      0.26      0.23        58\n        73.0       0.17      0.16      0.16        64\n        74.0       0.24      0.17      0.20        60\n        75.0       0.15      0.19      0.17        42\n        76.0       0.25      0.35      0.29        54\n        77.0       0.21      0.24      0.22        38\n        78.0       0.19      0.19      0.19        37\n        79.0       0.22      0.28      0.25        29\n        80.0       0.14      0.07      0.10        14\n        81.0       0.00      0.00      0.00        11\n       

In [69]:
importances = rf_model.feature_importances_
importances

array([0.03744721, 0.03178722, 0.03575491, 0.0404552 , 0.03625298,
       0.0410018 , 0.04889157, 0.04697089, 0.04227801, 0.04242649,
       0.02365178, 0.02869593, 0.03190028, 0.0317679 , 0.03444534,
       0.03714095, 0.02783372, 0.03556703, 0.03508971, 0.03344194,
       0.02794827, 0.02486208, 0.0249245 , 0.0329169 , 0.0630126 ,
       0.01995191, 0.00251682, 0.04210553, 0.00797916, 0.00537847,
       0.00563705, 0.00738568, 0.00562465, 0.00695552])

In [70]:
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.06301260174087588, 'FP'),
 (0.048891573380865985, 'PTS'),
 (0.046970887446489945, 'FGM'),
 (0.04242648583326893, 'FG%'),
 (0.04227800620539427, 'FGA'),
 (0.04210552997650525, '+/-'),
 (0.04100179811940057, 'MIN'),
 (0.0404552015657529, 'W'),
 (0.037447212801090035, 'Unnamed: 0'),
 (0.0371409480432119, 'FT%'),
 (0.036252983042426384, 'L'),
 (0.03575491405528425, 'GP'),
 (0.03556702574981331, 'DREB'),
 (0.03508971411602435, 'REB'),
 (0.03444533764553999, 'FTA'),
 (0.03344194223174741, 'AST'),
 (0.03291689505439702, 'PF'),
 (0.031900276944388725, '3P%'),
 (0.03178722328473721, 'AGE'),
 (0.03176789571595319, 'FTM'),
 (0.028695933481760715, '3PA'),
 (0.02794826786786121, 'TOV'),
 (0.027833719211925404, 'OREB'),
 (0.02492450043358263, 'BLK'),
 (0.02486208320506394, 'STL'),
 (0.023651780106834375, '3PM'),
 (0.019951908908776878, 'DD2'),
 (0.007979156652241911, 'SEASON_2014-15'),
 (0.00738568020342439, 'SEASON_2017-18'),
 (0.006955517341411289, 'SEASON_2019-20'),
 (0.0056370535550205325, '

In [ ]:
new_model = RandomForestClassifier(n_estimators=128, )